# Hausdorff- and Chamfer Distance 

Import custom files.

In [1]:
import area_comparison as ac
import general_functions as gf

Import other modules.

In [2]:
import os
import numpy as np
from skimage.draw import line, polygon
from scipy.spatial.distance import cdist
import glob
import cv2
import shapely

Variablen:

In [3]:
# Paths to contours that should be compared
path_Algo1 = '../outputs/pyRES'
name_ending_Algo1 = 'HSV.txt'

path_Algo2 = '../outputs/pyRES'
name_ending_Algo2 = 'Sobel.txt'

path_YOLO = '../outputs/yolov8seg/rcf'
name_ending_YOLO = 'YOLO.txt'

# Reference to compare with
path_ref = '../outputs/evaluation/rcf'
name_ending_ref = 'ur.txt'

# Path to the images
path_images = '../test_images/rcf'
name_ending_images = '.jpg'

Functions:

In [4]:
def hausdorff_distance(setA, setB):
    dists_AB = cdist(setA, setB)
    forward_hd = dists_AB.min(axis=1).max()
    backward_hd = dists_AB.min(axis=0).max()
    return max(forward_hd, backward_hd)

def chamfer_distance(setA, setB):
    dists = cdist(setA, setB)
    min_dists = np.min(dists, axis=1)
    return np.mean(min_dists), np.std(min_dists)

def chamfer_distance_symmetric(setA, setB):
    dists = cdist(setA, setB)  # (N x M) Matrix aller paarweisen Distanzen
    # Durchschnitt der kleinsten Distanzen von S1 zu S2
    term_1 = np.mean(np.min(dists, axis=1))
    # Durchschnitt der kleinsten Distanzen von S2 zu S1
    term_2 = np.mean(np.min(dists, axis=0))
    return term_1 + term_2

def load_yolo_contour(file_path, image_width, image_height):
    with open(file_path, 'r') as f:
        line_str = f.readline().strip()
        parts = line_str.split()[1:]
        points = np.array(list(map(float, parts))).reshape(-1, 2)
        points[:, 0] *= image_width
        points[:, 1] *= image_height
    return points

def densify_contour(contour):
    dense_points = []
    for i in range(len(contour)):
        x0, y0 = np.round(contour[i]).astype(int)
        x1, y1 = np.round(contour[(i + 1) % len(contour)]).astype(int)
        rr, cc = line(y0, x0, y1, x1)
        dense_points.extend(zip(cc, rr))
    return np.unique(np.array(dense_points), axis=0)

def get_contour_area_pixels(dense_contour, image_shape):
    rr, cc = polygon(dense_contour[:, 1], dense_contour[:, 0], shape=image_shape)
    return len(rr)

In [5]:
files_Reference = sorted([f for f in os.listdir(path_ref) if f.endswith(name_ending_ref)])
files_images = sorted([f for f in os.listdir(path_images) if f.endswith(name_ending_images)])

files_Algo1 = sorted([f for f in os.listdir(path_Algo1) if f.endswith(name_ending_Algo1)])
files_Algo2 = sorted([f for f in os.listdir(path_Algo2) if f.endswith(name_ending_Algo2)])
files_Yolo = sorted([f for f in os.listdir(path_YOLO) if f.endswith(name_ending_YOLO)])

assert len(files_Reference) == len(files_Algo1) == len(files_Algo1) == len(files_Yolo), "Dateienzahl in den Ordnern stimmt nicht überein!"

mean_over_all_Hausdorff_Algo1=[]
mean_over_all_Hausdorff_Algo2=[]
mean_over_all_Hausdorff_YOLO=[]
mean_over_all_Chamfer_Algo1=[]
mean_over_all_Chamfer_Algo2=[]
mean_over_all_Chamfer_YOLO=[]

#image_shape = (image_height, image_width)

# loop over all images
for i in range(0,len(files_images)):
    image_shape = cv2.imread(os.path.join(path_images, files_images[i])).shape[:2]
    image_width = image_shape[1]
    image_height = image_shape[0]
    
    filename = files_images[i]
    print(f"\n📄 Bild {i}: {filename}")

    # load contours
    contour_Rreference = load_yolo_contour(os.path.join(path_ref, files_Reference[i]), image_width, image_height)
    contour_Algo1 = load_yolo_contour(os.path.join(path_Algo1, files_Algo1[i]), image_width, image_height)
    contour_Algo2 = load_yolo_contour(os.path.join(path_Algo2, files_Algo2[i]), image_width, image_height)
    contour_Yolo = load_yolo_contour(os.path.join(path_YOLO, files_Yolo[i]), image_width, image_height)

    # build dense contours
    dense_contour_Reference = densify_contour(contour_Rreference)
    dense_contour_Algo1 = densify_contour(contour_Algo1)
    dense_contour_Algo2 = densify_contour(contour_Algo2)
    dense_contour_Yolo = densify_contour(contour_Yolo)

    # area of the countour in pixel
    area_pixels = get_contour_area_pixels(dense_contour_Reference, image_shape)

    # Hausdorff-Distances
    hausdorff_Algo1 = hausdorff_distance(dense_contour_Reference, dense_contour_Algo1)
    hausdorff_Algo2 = hausdorff_distance(dense_contour_Reference, dense_contour_Algo2)
    hausdorff_Yolo = hausdorff_distance(dense_contour_Reference, dense_contour_Yolo)

    #chamfer_distance_symmetric
    chamfer_Algo1_sym = chamfer_distance_symmetric(dense_contour_Reference, dense_contour_Algo1)
    chamfer_Algo2_sym = chamfer_distance_symmetric(dense_contour_Reference, dense_contour_Algo2)
    chamfer_YOLO_sym = chamfer_distance_symmetric(dense_contour_Reference, dense_contour_Yolo)
    
    mean_over_all_Hausdorff_Algo1.append(hausdorff_Algo1)
    mean_over_all_Hausdorff_Algo2.append(hausdorff_Algo2)
    mean_over_all_Hausdorff_YOLO.append(hausdorff_Yolo)
    mean_over_all_Chamfer_Algo1.append(chamfer_Algo1_sym)
    mean_over_all_Chamfer_Algo2.append(chamfer_Algo2_sym)
    mean_over_all_Chamfer_YOLO.append(chamfer_YOLO_sym)
    
    # output
    print(f"→ Hausdorff-Distanz (Algo1 vs Ref): {hausdorff_Algo1:.3f} px")
    print(f"→ Hausdorff-Distanz (Algo2 vs Ref): {hausdorff_Algo2:.3f} px")
    print(f"→ Hausdorff-Distanz (YOLO vs Ref): {hausdorff_Yolo:.3f} px")
    print(f"→ Chamfer-Distanz Algo1 (Sym):   {chamfer_Algo1_sym:.3f} px")
    print(f"→ Chamfer-Distanz Algo2 (Sym):   {chamfer_Algo2_sym:.3f} px")
    print(f"→ Chamfer-Distanz Yolo (Sym):   {chamfer_YOLO_sym:.3f} px")
    print(f"→ Area:   {area_pixels:.3f} px")
print('')
print('')
print(f"→ Mean over All Hausdorff-Distanz (Algo1 vs Ref): {np.mean(mean_over_all_Hausdorff_Algo1):.3f} {np.std(mean_over_all_Hausdorff_Algo1):.3f} px")
print(f"→ Mean over All Hausdorff-Distanz (Algo2 vs Ref): {np.mean(mean_over_all_Hausdorff_Algo2):.3f} {np.std(mean_over_all_Hausdorff_Algo2):.3f} px")
print(f"→ Mean over All Hausdorff-Distanz (YOLO vs Ref): {np.mean(mean_over_all_Hausdorff_YOLO):.3f} {np.std(mean_over_all_Hausdorff_YOLO):.3f} px")
print(f"→ Mean over All Chamfer-Distanz sym (Algo1 vs Ref): {np.mean(mean_over_all_Chamfer_Algo1):.3f} {np.std(mean_over_all_Chamfer_Algo1):.3f} px")
print(f"→ Mean over All Chamfer-Distanz sym (Algo2 vs Ref): {np.mean(mean_over_all_Chamfer_Algo2):.3f} {np.std(mean_over_all_Chamfer_Algo2):.3f} px")
print(f"→ Mean over All Chamfer-Distanz sym (YOLO vs Ref): {np.mean(mean_over_all_Chamfer_YOLO):.3f} {np.std(mean_over_all_Chamfer_YOLO):.3f} px")


📄 Bild 0: image2.jpg
→ Hausdorff-Distanz (Algo1 vs Ref): 1089.413 px
→ Hausdorff-Distanz (Algo2 vs Ref): 178.000 px
→ Hausdorff-Distanz (YOLO vs Ref): 173.000 px
→ Chamfer-Distanz Algo1 (Sym):   1157.581 px
→ Chamfer-Distanz Algo2 (Sym):   76.746 px
→ Chamfer-Distanz Yolo (Sym):   53.285 px
→ Area:   2144730.000 px

📄 Bild 1: image312.jpg
→ Hausdorff-Distanz (Algo1 vs Ref): 1788.034 px
→ Hausdorff-Distanz (Algo2 vs Ref): 1796.034 px
→ Hausdorff-Distanz (YOLO vs Ref): 53.852 px
→ Chamfer-Distanz Algo1 (Sym):   1716.959 px
→ Chamfer-Distanz Algo2 (Sym):   1730.021 px
→ Chamfer-Distanz Yolo (Sym):   19.716 px
→ Area:   1472643.000 px

📄 Bild 2: image425.jpg
→ Hausdorff-Distanz (Algo1 vs Ref): 575.438 px
→ Hausdorff-Distanz (Algo2 vs Ref): 35.693 px
→ Hausdorff-Distanz (YOLO vs Ref): 22.561 px
→ Chamfer-Distanz Algo1 (Sym):   853.654 px
→ Chamfer-Distanz Algo2 (Sym):   22.437 px
→ Chamfer-Distanz Yolo (Sym):   9.225 px
→ Area:   2168824.000 px

📄 Bild 3: image563.jpg
→ Hausdorff-Distanz (